In [3]:
import carla
import pygame
import time
import random
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState

# Class names for the steering angles
CLASS_NAMES = [
    "Turning Left Hard",
    "Turning Left Medium",
    "Turning Left Light",
    "No Turning",
    "Turning Right Light",
    "Turning Right Medium",
    "Turning Right Hard"
]

# model definition
class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(SteeringClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)


# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SteeringClassifier()
model.load_state_dict(torch.load("../Models/steering_model_final.pth", map_location=device))
model.eval().to(device)

image_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# pygame setup so that we can see the camera feed
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Manual Control")

# connect to CARLA
client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
blueprint_library = world.get_blueprint_library()

# spawn the vehicle and set its initial state so that it can be controlled with our model
vehicle_bp = blueprint_library.filter("vehicle.*model3*")[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

# spectator setup
spectator = world.get_spectator()

# placing camera for the car 'dash'
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '448')
camera_bp.set_attribute('image_size_y', '252')
camera_bp.set_attribute('fov', '145')
camera_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, cam_transform, attach_to=vehicle)

# Shared variables
camera_image = None
camera_np = None
pred_label = "Loading..."

def process_image(image):
    global camera_image, camera_np
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # RGB
    camera_np = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))

camera.listen(process_image)
clock = pygame.time.Clock()

print("Controls:")
print("Arrow keys: steer and drive")
print("Q/E: left/right turn signals")
print("ESC or close window: exit")

# main
try:
    while True:
        clock.tick(30)
        pygame.event.pump()

        keys = pygame.key.get_pressed()
        control = carla.VehicleControl()

        # manual driving using WASD control
        if keys[pygame.K_w]:
            control.throttle = 0.3
        elif keys[pygame.K_s]:
            control.brake = 0.5

        if keys[pygame.K_a]:
            control.steer = -0.5
        elif keys[pygame.K_d]:
            control.steer = 0.5

        vehicle.apply_control(control)

        # Handle signal lights
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            signal_indicator = "LEFT"
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            signal_indicator = "RIGHT"
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            signal_indicator = "NONE"

        #get turn current signal
        turn_signal = -1 if keys[pygame.K_q] else 1 if keys[pygame.K_e] else 0

        # predict class and steering angle
        if camera_np is not None:
            with torch.no_grad():
                img = image_transform(camera_np).unsqueeze(0).to(device)
                signal = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                output = model(img, signal.squeeze(1))
                pred_class = output.argmax(dim=1).item()
                pred_label = CLASS_NAMES[pred_class]

        # Update spectator camera
        car_transform = vehicle.get_transform()
        forward_vector = car_transform.get_forward_vector()
        cam_location = car_transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=car_transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # display camera feed with prediction so we can see
        if camera_image:
            screen.blit(camera_image, (0, 0))
            font = pygame.font.SysFont(None, 30)
            label_surface = font.render(f"Predicted: {pred_label}", True, (255, 255, 0))
            screen.blit(label_surface, (20, 20))
            pygame.display.flip()

        # for quit
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Exiting and cleaning up...")

finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


c:\Users\Pravin\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Pravin\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: '../Models/steering_model_final.pth'